In [ ]:
%pip install langchain openai pypdf

# Document Loading

In [1]:
import os
import sys

from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("./LawsofTheGame2022_23.pdf")
pages = loader.load()

In [2]:
len(pages)

230

In [3]:
trimmed_pages = pages[10:200] # Just the relevant pages
page = trimmed_pages[0]

In [4]:
print(page.page_content[0:500])

11
 Football is the greatest sport on earth. It is played on every continent, in every 
country and at many different levels. The fact that the Laws of the Game are the 
same for all football throughout the world, from the FIFA World Cup™ through 
to a game between young children in a remote village, is a considerable 
strength which must continue to be harnessed for the good of football 
everywhere.
 Football must have Laws which keep the game fair – this is a crucial foundation 
of the ‘beauti


# Document Splitting

In [5]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [6]:
docs = text_splitter.split_documents(trimmed_pages)

In [7]:
print("Length of trimmed_pages: ", len(trimmed_pages))
print("Length of splits: ", len(docs))

Length of trimmed_pages:  190
Length of splits:  245


In [8]:
print(docs[0])

page_content='11\n Football is the greatest sport on earth. It is played on every continent, in every \ncountry and at many different levels. The fact that the Laws of the Game are the \nsame for all football throughout the world, from the FIFA World Cup™ through \nto a game between young children in a remote village, is a considerable \nstrength which must continue to be harnessed for the good of football \neverywhere.\n Football must have Laws which keep the game fair – this is a crucial foundation \nof the ‘beautiful game’ and a vital feature of the ‘spirit’ of the game. The best \nmatches are those where the referee is rarely needed because the players play \nwith respect for each other, the match officials and the Laws.\n Football’s Laws are relatively simple compared to most other team sports, but \nas many situations are subjective and match officials are human, some \ndecisions will inevitably be wrong or cause debate and discussion. For some \npeople, this discussion is part o

# Embeddings
Let's take our splits and embed them.

In [9]:
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [10]:
from langchain.embeddings.openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [ ]:
#%pip install chromadb

In [ ]:
#!rm -rf ./docs/chroma

In [ ]:
#persist_directory = 'docs/chroma/'

In [ ]:
#%pip install tiktoken

In [ ]:
%pip install -U pinecone-client

# Vector Database

In [14]:
index_name = 'langchain-retrieval-agent-fast'

In [11]:
import pinecone      

pinecone.init(      
	api_key='6bf98108-9cd3-4960-af38-8e7cad11a1dd',      
	environment='us-east-1-aws'      
)      
index = pinecone.Index('langchain-retrieval-agent-fast')

/Users/ibrahimsaidi/Desktop/Builds/Kasuku/RefGPT/venv/lib/python3.11/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [12]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [15]:
from langchain.vectorstores import Pinecone

# IMPORTANT: DO THIS ONLY ONCE

# The OpenAI embedding model `text-embedding-ada-002 uses 1536 dimensions`
#vectorstore = Pinecone.from_documents(docs, embeddings, index_name=index_name)

In [16]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 245}},
 'total_vector_count': 245}

# Initializing the Conversational Agent
Our conversational agent needs a Chat LLM, conversational memory, and a RetrievalQA chain to initialize. We create these using:

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA

# chat completion llm
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)
# conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)
# retrieval qa chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

Using these we can generate an answer using the run method:

In [18]:
query = "how long does a match last for?"

In [19]:

qa.run(query)

'A match lasts for two equal halves of 45 minutes each, totaling 90 minutes. However, the duration of the match can be reduced if agreed upon by the referee and the two teams before the start of the match and in accordance with competition rules.'

But this isn't yet ready for our conversational agent. For that we need to convert this retrieval chain into a tool. We do that like so:

In [20]:
from langchain.agents import Tool

tools = [
    Tool(
        name='Knowledge Base',
        func=qa.run,
        description=(
            'use this tool when answering general knowledge queries to get '
            'more information about the topic'
        )
    )
]

Now we can initialize the agent like so:

In [21]:
from langchain.agents import initialize_agent

agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

In [22]:
agent(query)



> Entering new AgentExecutor chain...
{
    "action": "Knowledge Base",
    "action_input": "duration of a match"
}
Observation: A match typically lasts for two equal halves of 45 minutes each, making a total of 90 minutes. However, the duration of the match can be reduced if agreed upon by the referee and the teams before the start of the match, and if it is in accordance with competition rules.
Thought:{
    "action": "Final Answer",
    "action_input": "A match typically lasts for two equal halves of 45 minutes each, making a total of 90 minutes. However, the duration of the match can be reduced if agreed upon by the referee and the teams before the start of the match, and if it is in accordance with competition rules."
}

> Finished chain.


{'input': 'how long does a match last for?',
 'chat_history': [],
 'output': 'A match typically lasts for two equal halves of 45 minutes each, making a total of 90 minutes. However, the duration of the match can be reduced if agreed upon by the referee and the teams before the start of the match, and if it is in accordance with competition rules.'}

Looks great, now what if we ask it a non-general knowledge question?

In [23]:
agent("what is 2 * 7?")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "The product of 2 multiplied by 7 is 14."
}

> Finished chain.


{'input': 'what is 2 * 7?',
 'chat_history': [HumanMessage(content='how long does a match last for?', additional_kwargs={}, example=False),
  AIMessage(content='A match typically lasts for two equal halves of 45 minutes each, making a total of 90 minutes. However, the duration of the match can be reduced if agreed upon by the referee and the teams before the start of the match, and if it is in accordance with competition rules.', additional_kwargs={}, example=False)],
 'output': 'The product of 2 multiplied by 7 is 14.'}

Perfect, the agent is able to recognize that it doesn't need to refer to it's general knowledge tool for that question. Let's try some more questions.

In [26]:
agent(
    " How many players are allowed to play at once? "
)



> Entering new AgentExecutor chain...
{
    "action": "Knowledge Base",
    "action_input": "How many players are allowed to play at once in what context? Could you please provide more information?"
}
Observation: In the context of a regular match, each team is allowed to have a maximum of eleven players on the field, including one goalkeeper. However, a match may not start or continue if either team has fewer than seven players. If a team deliberately has fewer than seven players, the referee is not obliged to stop play, but the match must not resume after the ball has gone out of play if a team does not have the minimum number of seven players.

In official competitions, the number of substitutes that can be used in a match is determined by FIFA, the confederation, or the national football association. For competitions involving the top division clubs or senior international teams, where a maximum of five substitutes is permitted, each team has a maximum of three substitution oppor

{'input': ' How many players are allowed to play at once? ',
 'chat_history': [HumanMessage(content='how long does a match last for?', additional_kwargs={}, example=False),
  AIMessage(content='A match typically lasts for two equal halves of 45 minutes each, making a total of 90 minutes. However, the duration of the match can be reduced if agreed upon by the referee and the teams before the start of the match, and if it is in accordance with competition rules.', additional_kwargs={}, example=False),
  HumanMessage(content='what is 2 * 7?', additional_kwargs={}, example=False),
  AIMessage(content='The product of 2 multiplied by 7 is 14.', additional_kwargs={}, example=False)],
 'output': 'In a regular match, each team is allowed to have a maximum of eleven players on the field, including one goalkeeper.'}